## Find a Position in a Genome Minimizing the Skew 


In [ ]:
def skew(genome):
    val = 0;
    track = []
    track.append(0)
    for i in genome:
        if i == "G":
            val += 1
        elif i == "C":
            val -=1
        track.append(val)

    # for k in track:
    #     print(k, end=" ")
    return track
skew("GAGCCACCGCGATA")
def skew_minima(genome):
    track = skew(genome)
    idx_list = []
    # print(track)
    minima = min(track)
    for i in range(len(track)):
        if (track[i] == minima):
            idx_list.append(i)
    for i in idx_list:
        print(i,end=" ")
skew_minima("")

## Find Patterns Forming Clumps in a String

In [ ]:
def NumberToPattern(index, k):
    table = {0:'A', 1:'C', 2:'G', 3:'T'}
    if k == 1:
        # print("ret for k: ",k ," index : ",int(index) , " => " ,table[int(index)])
        return table[int(index)]
    prefixIndex = index / 4
    r = index % 4
    prefixPattern = NumberToPattern(prefixIndex, k-1)
    symbol = table[int(r)]
    # print("ret for k: ",k ," index : ",int(index) , " => " , prefixPattern + symbol)
    return prefixPattern + symbol

def SymbolToNumber(symbol):
    if symbol=='A':
        return 0
    elif symbol=='C':
        return 1
    elif symbol=='G':
        return 2
    else:
        return 3
    
def PatternToNumber(pattern):
    if pattern=='':
        return 0
    symbol=pattern[-1]
    prefix=pattern[:-1]
    return 4*PatternToNumber(prefix)+SymbolToNumber(symbol)

def FrequenceArray(sequence, k):
    FrequencyArray = []
    for i in range(4**k):
        FrequencyArray.append(0)
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        index = PatternToNumber(kmer) #This is for hashing the kmer sequence
        # print("FA : " , FrequencyArray[index])
        FrequencyArray[index] = FrequencyArray[index] + 1
    return FrequencyArray

def ClumpFinding(Genome, k, t, L):
    clumps = []
    FrequencePattern = set()
    for i in range(4**k - 1):
        clumps.append(0)
    for i in range(len(Genome) - L):
        window = Genome[i:i+L]
        FrequenceArray = FrequenceArray(window, k)
        for j in range(4**k - 1):
            if FrequenceArray[j] >= t:
                clumps[j] = 1
    for i in range(4**k - 1):
        if clumps[i] == 1:
            pattern = NumberToPattern(i, k)
            FrequencePattern.add(pattern)
    return FrequencePattern

## Find a Median String 

In [ ]:
def HammingDistance(p, q):
    count = 0
    for (i,j) in zip(p,q):
        if i != j:
            count += 1
    return count

def getKmers(dna, k):
    for i in range(len(dna) - k + 1):
        yield dna[i:i+k]

def DiffBetweenPatternandStrings(pattern, dna): # minimizes the sum of hamming distances to the given dnas with a pattern
    k =  len(pattern)
    diff = 0;
    for d in dna:
        ham_distance = 2 ** 31
        for i in range (len(d) - k + 1):
            pattern_ = d[i : i+k]
            if (HammingDistance(pattern,pattern_) < ham_distance):
                ham_distance = HammingDistance(pattern,pattern_)
        diff = diff + ham_distance

    return diff

def MedianString(dna, k) : # A string that minimizes the sum of distances to the strings of a given set 
    diff = 2 ** 31 
    kmers = generate_strings(k)
    median = ""
    for kmer in kmers:
        temp_diff =  DiffBetweenPatternandStrings(kmer, dna)
        if (temp_diff < diff) :
            diff =  temp_diff
            median = kmer

    return median

## Find a Profile-most Probable k-mer in a String

In [4]:
def calculateProbability(pattern, profile_mat): # given a profile, calculates the probability of a string occuring 
    temp_pr = 1.0
    for j in range(len(pattern)):
        if(pattern[j] == "A"):
            temp_pr  = temp_pr * profile_mat ["A"][j]
        elif(pattern[j] == "C"):
            temp_pr  = temp_pr * profile_mat ["C"][j]
        elif(pattern[j] == "G"):
            temp_pr  = temp_pr * profile_mat ["G"][j]
        elif(pattern[j] == "T"):
            temp_pr  = temp_pr * profile_mat ["T"][j]
    return temp_pr


def ProfileMostProbableKmer(text, k, profile):
    max_p = -1
    most_probable_kmer = ''
    for i in range(len(text)-k+1):
        pattern = text[i: i+k]
        p = calculateProbability(pattern, profile)
        if p > max_p:
            max_p = p
            most_probable_kmer = pattern           
    return most_probable_kmer

In [32]:

def Count(Motifs):
    count = {} # initializing the 4xk count dictionary
    k = len(Motifs[0])
    count = {'A':[0]*k,'C':[0]*k,'G':[0]*k,'T':[0]*k}
    
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def Consensus(Motifs): # get the consensus string for given motifs by picking the most frequent symbol in each column
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

def Profile(Motifs): ## generate profile matrix from motifs
    t = len(Motifs)
    k = len(Motifs[0])
    profile = Count(Motifs)
    print("Count: \n", profile)

    for i in profile:  
        for j in range(k):
           profile[i][j] = float(profile[i][j])/float(t)   
    return profile

def Score(Motifs):
    consensus = Consensus(Motifs) #consensus string
    counts = Count(Motifs) # count matrix
    score = len(Motifs)*len(consensus) 
    i = 0
    for symbol in consensus:  
        score -= counts[symbol][i] 
        i += 1
    return score 

def GreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k]) # first k-mers from each of the dna strings
    
    n = len(Dna[0]) 
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k]) # kmer from 1st string
        for j in range(1, t): #for rest of the strings
            P = Profile(Motifs[0:j]) # successively calculates profile matrix by adding one string per iteration
            print("intmed Profile for ", Motifs[0:j])
            print(P)
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))
            print("PMP kmer for ",Dna[j]," is: ",ProfileMostProbableKmer(Dna[j], k, P))

        print("current motifs: ", Motifs)
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
        print("..........................................................................................")
    return BestMotifs

In [33]:
GreedyMotifSearch(["ACCTAT", "AGTTGG", "CCGTTC", "ACGTAA"], 4, 4)

Count: 
 {'A': [1, 0, 0, 0], 'C': [0, 1, 1, 0], 'G': [0, 0, 0, 0], 'T': [0, 0, 0, 1]}
intmed Profile for  ['ACCT']
{'A': [1.0, 0.0, 0.0, 0.0], 'C': [0.0, 1.0, 1.0, 0.0], 'G': [0.0, 0.0, 0.0, 0.0], 'T': [0.0, 0.0, 0.0, 1.0]}
PMP kmer for  AGTTGG  is:  AGTT
Count: 
 {'A': [2, 0, 0, 0], 'C': [0, 1, 1, 0], 'G': [0, 1, 0, 0], 'T': [0, 0, 1, 2]}
intmed Profile for  ['ACCT', 'AGTT']
{'A': [1.0, 0.0, 0.0, 0.0], 'C': [0.0, 0.5, 0.5, 0.0], 'G': [0.0, 0.5, 0.0, 0.0], 'T': [0.0, 0.0, 0.5, 1.0]}
PMP kmer for  CCGTTC  is:  CCGT
Count: 
 {'A': [2, 0, 0, 0], 'C': [1, 2, 1, 0], 'G': [0, 1, 1, 0], 'T': [0, 0, 1, 3]}
intmed Profile for  ['ACCT', 'AGTT', 'CCGT']
{'A': [0.6666666666666666, 0.0, 0.0, 0.0], 'C': [0.3333333333333333, 0.6666666666666666, 0.3333333333333333, 0.0], 'G': [0.0, 0.3333333333333333, 0.3333333333333333, 0.0], 'T': [0.0, 0.0, 0.3333333333333333, 1.0]}
PMP kmer for  ACGTAA  is:  ACGT
current motifs:  ['ACCT', 'AGTT', 'CCGT', 'ACGT']
.....................................................

['ACCT', 'AGTT', 'CCGT', 'ACGT']